In [7]:
using Pkg
Pkg.add("StatsPlots")

  Updating registry at `C:\Users\masahiro\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % [===>                                     ]  5.7 %                                    ]  9.8 %.6 %                             ]  25.1 %==============>                          ]  32.6 %6 %.6 %>                 ]  55.3 %>           ]  70.5 %.3 %89.0 % [======================================>  ]  94.3 % % Resolving package versions...
 Installed AxisAlgorithms ───────── v1.0.0
 Installed DataValueInterfaces ──── v1.0.0
 Installed KernelDensity ────────── v0.5.1
 Installed OffsetArrays ─────────── v0.11.1
 Installed StatsPlots ───────────── v0.12.0
 Installed FFTW ─────────────────── v1.0.1
 Installed DataValues ───────────── v0.4.12
 Installed Optim ────────────────── v0.19.3
 Installed WoodburyMatrices ─────── v0.4.1
 Installed Widgets ──────────────── v0.6.2
 Installed AbstractFFTs ─

In [8]:
using Distributions, StatsPlots, Random, Plots; gr();

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1242


In [2]:
# Data from the 1975 World Almanac of height (dataX) vs weight( dataY)
dataX = collect(58:72)
dataY = [115, 117, 120, 123, 126, 129, 132, 135, 139, 142, 146, 150, 154, 159, 164];

In [3]:
# Quick plot of the data
scatter(dataX,dataY,label="",xlabel="Height (in)", ylabel="Weight (lbs)")

UndefVarError: UndefVarError: scatter not defined

In [4]:
# A model for the data - this is assumed
f(x,p1,p2,p3) = p1 + p2*(x/12) + p3*(x/12).^2;

In [5]:
# The parameters for our model are defined as random variables.
# This information is being given from on-high.
meanP = [261.8, -88.1, 11.96];
varP = [778.35, 106.61, 0.91];
stdP = sqrt.(varP)

3-element Array{Float64,1}:
 27.898924710461515 
 10.325211862233143 
  0.9539392014169457

In [6]:
# Example of the model output
f(60,meanP[1],meanP[2],meanP[3])

120.30000000000001

In [ ]:
# Plot the model and data together
scatter(dataX,dataY,label="Data",xlabel="Height (in)", 
    ylabel="Weight (lbs)")
plot!(dataX,f.(dataX,meanP[1],meanP[2],meanP[3]),
    label="Model",legend=:topleft,linewidth=3)

In [ ]:
# Define the distributions of the parameters as normal (assumed)
p1_dist = Normal(meanP[1],stdP[1]) 
p2_dist = Normal(meanP[2],stdP[2]) 
p3_dist = Normal(meanP[3],stdP[3])

In [ ]:
# Collect N samples of the disjoint and independent distributions
N = 1000
rand_params = 
[ rand(p1_dist, N) rand(p2_dist, N) rand(p3_dist, N)]'

In [ ]:
# Evaluate the model N times (samples above) and plot them
plt = plot(xlabel="Height (in)", ylabel="Weight (lbs)",ylimits=(0,300))
for i = 1:N
    plot!(plt, 
        dataX,f.(dataX, rand_params[1,i],rand_params[2,i],rand_params[3,i]),
        label="",color=:black,alpha=0.1,legend=:topleft)
end
scatter!(dataX,dataY,label="Data",
    xlabel="Height (in)", ylabel="Weight (lbs)")
plt

In [ ]:
# Just pull out a single time point and plot the histogram
QoI = zeros(N)
for i = 1:N
    QoI[i] = f(65.0, rand_params[1,i],rand_params[2,i],rand_params[3,i])
end

In [ ]:
histogram(QoI)

In [7]:
mean(QoI)

UndefVarError: UndefVarError: QoI not defined

In [8]:
# This mean matches the sampling mean above
f(65.0,meanP[1],meanP[2],meanP[3])

135.50138888888898

In [9]:
var(QoI)

UndefVarError: UndefVarError: QoI not defined

In [ ]:
# This variance matches the sampling variance above
varP[1] + (65.5/12)^2 * varP[2] + (65.5/12)^4 * varP[3]

In [ ]:
# Define functions for these computed values and plot them
meanF(x) = f(x,meanP[1],meanP[2],meanP[3])
varF(x) = varP[1] + (x/12)^2 * varP[2] + (x/12)^4 * varP[3];

In [ ]:
scatter(dataX,dataY,label="Data",xlabel="Height (in)", ylabel="Weight (lbs)",ylimits=(0,300))
plot!(dataX,meanF.(dataX),label="Mean F", color=:red,linewidth=3,legend=:topleft)

In [ ]:
scatter(dataX,dataY,label="Data",xlabel="Height (in)", ylabel="Weight (lbs)",ylimits=(-20,300))
plot!(dataX,meanF.(dataX),label="Mean F", color=:red,linewidth=3,legend=:topleft)
plot!(dataX,meanF.(dataX) + 2 .* sqrt.(varF.(dataX)),label="Mean F + 2STD", line=(3,:dot,:blue),legend=:topleft)
plot!(dataX,meanF.(dataX) - 2 .* sqrt.(varF.(dataX)),label="Mean F - 2STD", line=(3,:dot,:blue),legend=:topleft)

In [ ]:
# This covariance matrix is being given from on-high.
V = [778.35 -287.93 26.51; -287.93 106.61 -9.82; 26.51 -9.82 0.91]

In [ ]:
# We need the LA package to do the following checks
using LinearAlgebra

In [ ]:
# Covariance matricies are always hermitian (e.g., symmetric)
ishermitian(V)

In [ ]:
# Covariance matricies are always positive definite
isposdef(V)

In [ ]:
# Define a multivariate normal that includes the covariance
pdist = MvNormal(meanP,V)

In [ ]:
# Collect N samples from the now DEPENDENT parameters
N = 1000
rand_params = rand(pdist,N)

In [ ]:
# Run the model for N samples and plot
plt = scatter(xlabel="Height (in)", ylabel="Weight (lbs)",ylimits=(0,300))
for i = 1:N
    plot!(plt, dataX,f.(dataX, rand_params[1,i],rand_params[2,i],rand_params[3,i]),label="",
        color=:black,alpha=0.1,legend=:topleft)
end
scatter!(dataX,dataY,label="Data",xlabel="Height (in)", ylabel="Weight (lbs)")
plt

In [ ]:
# Extract a single time point and show mean/variance are same
QoI = zeros(N)
for i = 1:N 
    QoI[i] = f(65.0, rand_params[1,i],rand_params[2,i],rand_params[3,i])
end

In [ ]:
mean(QoI)

In [ ]:
meanF(65.0)

In [ ]:
var(QoI)

In [ ]:
# The direct calculation of variance is more complicated because
# of the covariance, but still possible to compute.
varP[1] + (65.0/12)^2 * varP[2] + (65.0/12)^4 * varP[3] + 
    2 * ( (65.0/12) * V[1,2] + (65.0/12)^2 * V[1,3] + (65.0/12)^3 * V[2,3] )

In [ ]:
meanF(x) = f(x,meanP[1],meanP[2],meanP[3])
varF(x) = varP[1] + (x/12)^2 * varP[2] + (x/12)^4 * varP[3] + 
    2 * ( (x/12) * V[1,2] + (x/12)^2 * V[1,3] + (x/12)^3 * V[2,3] );

In [ ]:
scatter(dataX,dataY,label="Data",xlabel="Height (in)", ylabel="Weight (lbs)",ylimits=(0,300))
plot!(dataX,meanF.(dataX),label="Mean F", color=:red,linewidth=3,legend=:topleft)

In [ ]:
scatter(dataX,dataY,label="Data",xlabel="Height (in)", ylabel="Weight (lbs)")#,ylimits=(0,300))
plot!(dataX,meanF.(dataX),label="Mean F", color=:red,linewidth=3,legend=:topleft)
plot!(dataX,meanF.(dataX) + sqrt.(varF.(dataX)),label="Mean F + 1STD", line=(3,:dot,:blue),legend=:topleft)
plot!(dataX,meanF.(dataX) - sqrt.(varF.(dataX)),label="Mean F - 1STD", line=(3,:dot,:blue),legend=:topleft)

In [ ]:
scatter(dataX,dataY,label="Data",xlabel="Height (in)", ylabel="Weight (lbs)")
plot!(dataX,meanF.(dataX),label="Mean F", color=:red,linewidth=3,legend=:topleft)
plot!(dataX,meanF.(dataX) + sqrt.(varF.(dataX)),label="Mean F + 1STD", line=(3,:dot,:blue),legend=:topleft)
plot!(dataX,meanF.(dataX) - sqrt.(varF.(dataX)),label="Mean F - 1STD", line=(3,:dot,:blue),legend=:topleft)